In [ ]:
from transformers import (AutoTokenizer, AutoModelForCausalLM)
import transformers
import torch
transformers.logging.set_verbosity_error()
from hotmoe import HotmoeModel

checkpoint_path = ''  # replace with your checkpoint path
model_name_or_path = '' # replace with your base model name or path
base_model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.padding_side == 'right':
    tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = HotmoeModel.from_pretrained(base_model, checkpoint_path)
peft_weights = torch.load(checkpoint_path + '/' + 'adapter_model.safetensors')
model.load_state_dict(peft_weights, strict=False)
model.eval()
model

In [ ]:
question = 'Please choose the correct ending to complete the given sentence: Canoeing: Two women in a child are shown in a canoe while a man pulls the canoe while standing in the water, with other individuals visible in the background. the child and a different man\n\nEnding1: are then shown paddling down a river in a boat while a woman talks. Ending2: are driving the canoe, they go down the river flowing side to side. Ending3: sit in a canoe while the man paddles. Ending4: walking go down the rapids, while the man in his helicopter almost falls and goes out of canoehood.\n\nAnswer format: ending1/ending2/ending3/ending4'
input_text = f"<|im_start|>system\nYou are a helpful assistant.\n<|im_end|>\n<|im_start|>human\n{question}\n<|im_end|>\n<|im_start|>assistant\n"

In [ ]:
ids = tokenizer(input_text, return_tensors='pt', add_special_tokens=False)
res = model.generate(input_ids=ids.input_ids, max_new_tokens=128, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(res[0]).replace(input_text, '').replace('<|endoftext|>', ''))